# Spotify Streaming Pattern Analysis

Author: Arun Ganapathy <br>
Dated: 9th December 2022

In this project, we aim to analyze Billboard chart data and Spotify track information to gain insights into popular music trends. The project involves the following key steps:

1. **Data Collection**: We retrieve historical Billboard chart data for pop songs from 2008 to 2021. This data includes details about artists, track titles, and years of charting.

2. **Data Cleaning and Transformation**: We preprocess the collected data by cleaning it, converting data types, and handling any discrepancies.

3. **Exploratory Analysis**: We explore the dataset to identify patterns, trends, and interesting insights related to artists, genres, and charting over the years.

4. **Genre Correction**: We correct and standardize the genre information using text processing techniques.

5. **Integration with Spotify Data**: We integrate Spotify track information with the Billboard data to enhance our analysis. This includes identifying popular artists and tracks based on Spotify metrics.

6. **Data Export**: After processing and analyzing the data, we export the final dataset to a CSV file for further use.

Through this project, we aim to uncover how music popularity has evolved over the years, identify dominant genres, and highlight noteworthy artists and tracks within the given time frame.


In [1]:
import billboard
import pandas as pd
from tqdm import tqdm

In [2]:
# Specify the chart type ('pop-songs') and the year (2008) for the ChartData.
# This will retrieve Billboard chart data for the specified year and chart type.
chart = billboard.ChartData('pop-songs', year=2008)

In [3]:
# Create an empty DataFrame called 'df'.
df = pd.DataFrame()

# Add two empty columns 'Artist' and 'Title' to the DataFrame.
df[['Artist', 'Title']] = ""

# Add a new column 'Year' with default value 0 to the DataFrame.
df['Year'] = 0

In [4]:
# Loop through years from 2008 to 2021 (inclusive).
for i in tqdm(range(2008, 2022)):
    # Retrieve Billboard chart data for 'pop-songs' chart and the current year (i).
    chart = billboard.ChartData('pop-songs', year=i)
    
    # Loop through the entries in the current year's chart.
    for j in range(len(chart)):
        # Append a new row to the DataFrame 'df' with artist, title, and year information.
        df.loc[len(df)] = [chart.entries[j].artist, chart.entries[j].title, i]

100%|██████████| 14/14 [00:06<00:00,  2.14it/s]


In [5]:
# Export the DataFrame 'df' to a CSV file named 'billboard_08_21.csv'. # this billboard data is later used during visual development
df.to_csv('billboard_08_21.csv')

## Data Cleaning

In [6]:
# Read in Spotify data for cleaning
data = pd.read_csv('final_data.csv')
data.head()

,track_name,genre,artist_name,track_id,year,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,I Like It,Rap,Cardi B,58q2HKrzhC3ozto2nDdN4z,2010,90,0.099,0.816,253390,0.726,0.0,F,0.3720,-3.998,Minor,0.1290,136.048,44655,0.650
1,I Like It,Pop,Cardi B,58q2HKrzhC3ozto2nDdN4z,2010,90,0.099,0.816,253390,0.726,0.0,F,0.3720,-3.998,Minor,0.1290,136.048,44655,0.650
2,Baby,Rap,Amenazzy,2IA8cFA46DkjgbsRG7kKbp,2010,86,0.327,0.700,235973,0.710,0.0,B,0.1290,-4.835,Major,0.0532,167.971,44655,0.582
3,Baby,Reggaeton,Amenazzy,2IA8cFA46DkjgbsRG7kKbp,2010,86,0.327,0.700,235973,0.710,0.0,B,0.1290,-4.835,Major,0.0532,167.971,44655,0.582
4,"Hey, Soul Sister",Pop,Train,4HlFJV71xXKIGcU3kRyttv,2010,83,0.185,0.673,216773,0.886,0.0,C#,0.0826,-4.440,Minor,0.0431,97.012,44655,0.795


In [7]:
# Converting specific columns in the 'data' DataFrame to string data type.
# This ensures consistent data type for 'track_name', 'genre', and 'artist_name' columns.
data = data.astype({"track_name": str, "genre": str, "artist_name": str})

In [8]:
# Filtering out rows in the 'data' DataFrame where the 'track_name' column contains "?".
# The tilde (~) operator is used to negate the condition, keeping only rows without "?".
data = data[~data.track_name.str.contains("?", regex=False)]

In [9]:
# Grouping the 'data' DataFrame by unique combinations of 'track_name' and 'artist_name'.
# The 'first()' function is applied to each group, retaining the first occurrence within each group.
# Finally, the 'reset_index()' function is used to reset the index and create a new DataFrame.
data = data.groupby(['track_name', 'artist_name']).first().reset_index()

In [10]:
# Filtering the 'data' DataFrame to select rows where the 'artist_name' column is 'Cardi B'.
cardi_b_data = data[data.artist_name == 'Cardi B']
cardi_b_data.head()

,track_name,artist_name,genre,track_id,year,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
198,Bodak Yellow,Cardi B,Rap,6KBYefIoo7KydImq1uUQlL,2017,81,0.0659,0.926,223713,0.703,0.0,B,0.231,-6.337,Minor,0.103,125.022,44655,0.485
677,I Like It,Cardi B,Rap,58q2HKrzhC3ozto2nDdN4z,2010,90,0.0990,0.816,253390,0.726,0.0,F,0.372,-3.998,Minor,0.129,136.048,44655,0.650


In [11]:
# Replacing occurrences of 'Children’s Music' with 'Children's Music' in the 'genre' column of the 'data' DataFrame.
data.genre = data.genre.str.replace('Children’s Music', "Children's Music")

In [12]:
# Importing the 're' module for working with regular expressions.
import re

# Looping through each row in the 'data' DataFrame using the 'iterrows()' function.
for i, row in data.iterrows():
    # Checking if the 'genre' column in the current row is equal to 'Reggae'.
    if data.loc[i, 'genre'] == 'Reggae':
        # Replacing the 'genre' value with 'Reggaeton' for the current row.
        data.loc[i, 'genre'] = 'Reggaeton'

In [13]:
print(set(data.genre))

{'Movie', 'Soundtrack', 'World', 'Reggaeton', 'Anime', 'Hip-Hop', 'Folk', 'Electronic', 'Country', 'Jazz', 'Blues', 'Classical', 'Comedy', 'Rap', 'Ska', 'A Capella', 'Soul', 'Dance', 'R&B', "Children's Music", 'Opera', 'Alternative', 'Indie', 'Pop', 'Rock'}


In [14]:
data.head(5)

,track_name,artist_name,genre,track_id,year,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,#Beautiful,Mariah Carey,R&B,617KSbx52ACbnQBxSsG26X,2013,55,0.34600,0.677,199867,0.749,0.000000,E,0.3470,-5.405,Major,0.0391,107.042,44655,0.469
1,#SELFIE,The Chainsmokers,Dance,1HOlb9rdNOmy9b1Fakicjo,2014,65,0.01340,0.807,183750,0.916,0.000012,C,0.0787,-3.282,Major,0.2260,127.973,44655,0.651
2,#thatPOWER,will.i.am,Pop,01TuObJVd7owWchVRuQbQw,2013,67,0.00112,0.797,279507,0.608,0.000077,F#,0.0748,-6.096,Minor,0.0583,128.000,44655,0.403
3,1+1,Beyoncé,R&B,1pzJboOZaDNwshBnOlNh3a,2011,62,0.38100,0.303,273880,0.384,0.000005,F#,0.0661,-7.342,Major,0.0339,63.345,44656,0.261
4,24K Magic,Bruno Mars,Pop,6b8Be6ljOzmkOmFslEb23P,2017,81,0.03400,0.818,225983,0.803,0.000000,C#,0.1530,-4.282,Major,0.0797,106.970,44655,0.632


In [15]:
# Exporting the 'data' DataFrame to a CSV file named 'spotify_data.csv'.
# The 'index' parameter is set to False to exclude the index column from the CSV.
data.to_csv('spotify_data.csv', index=False)

### End of data processing